# Google plam LLM projejct 
* bulding an llm  based on the customer most answered questions

In [1]:
!pip install "numpy<2"
!pip install "sentence-transformers==2.2.2"
!pip install "InstructorEmbedding>=1.0.1"
!pip install "langchain>=0.2.0"
!pip install "langchain-community>=0.0.11"
!pip install "torch>=2.0"
!pip install -U "sentence-transformers==2.2.2"
!pip install -U "InstructorEmbedding>=1.0.1"
!pip install sentence-transformers



  Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl (20.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 4.1.0
    Uninstalling sentence-transformers-4.1.0:
      Successfully uninstalled sentence-transformers-4.1.0
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
Using cached packaging-24.2-py3-none-any.whl (65 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [2]:
import pandas as pd 
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt 
import seaborn as sns
from langchain_google_genai import ChatGoogleGenerativeAI 
from langchain.prompts import PromptTemplate

import re

In [3]:
import pprint
import google.generativeai as palm

/Users/phulusoravhuhali/Documents/phuluso_all/phuluso_data_science/Data_science/mine/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

load_dotenv()

True

In [5]:
api_key = os.environ.get("api_key")


In [6]:
palm.configure(api_key=api_key)

In [7]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


In [8]:

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api_key,
    temperature=0.9,

)


In [9]:

prompt = "can i give you a df and you give me a query "
response = llm.invoke(prompt)
print(response.content) 



Yes, I can.  Please provide the DataFrame.  To make it easy for me to understand and give you the best query, please also tell me:

* **What is the DataFrame about?** (e.g., customer orders, sensor readings, website traffic)  A brief description helps me understand the context.
* **What information are you trying to extract?** (e.g., total sales for a specific month, average sensor readings, top 10 website pages)  This is the most important part. Be as specific as possible.
* **How should the data be presented?** (e.g., aggregated values, a list of specific rows, a table)
* **What format is your DataFrame in?** (e.g.,  a list of dictionaries, a Pandas DataFrame representation in Python, CSV data, etc).  If it's a Pandas DataFrame, please include the column names. If it's another format, please specify the format and structure clearly.


The more details you provide, the better I can assist you.


In [10]:
dc  = pd.read_csv("customer_support_tickets.csv")
dc['extracted_question'] = dc['Ticket Description'].str.extract(r'(\?.*$)')
dc['cleaned_question'] = dc['extracted_question'].str.lower().str.strip()
dc = dc.dropna(subset=['cleaned_question'])


In [11]:
text = " ".join(dc['extracted_question'] .astype(str).tolist())
text = re.sub(r"[^A-Za-z\s]", "", text)
text = text.lower()


In [12]:
prompt = f"in a column called Ticket Description in {text} can you give me 5 most asked quesions just give me 1 to 5  simple answers, keep it simple  "
response = llm.invoke(prompt)
print(response.content) 

Here's a breakdown of the customer feedback, categorized for easier analysis.  I've also included 5 frequently asked questions based on the data, with simple answers.


**Categorized Customer Feedback:**

* **Charging Issues:** The original charger isn't charging the product properly.  This is mentioned repeatedly.

* **Software/Firmware Problems:**  A recent firmware update caused issues for many users.  The problem is often intermittent, sometimes working fine, and other times malfunctioning unexpectedly.

* **Functionality Issues:** Users can't find certain options or perform desired actions within the product.  Specific features or apps cause consistent problems.

* **Error Messages:**  A recurring "errormessage" pops up on the product screen.

* **Battery Life:**  Significant decrease in battery life reported.

* **Security Concerns:**  Users are concerned about the security of their data and want assurance it's safe.

* **Unresolved Customer Support:** Many users report contactin

# Using langchain

In [13]:
from langchain.document_loaders.csv_loader import CSVLoader

In [14]:
load = CSVLoader(file_path="customer_support_tickets.csv",source_column="Ticket Description")
df  = load.load()

### testing embedings 

pip install --upgrade --force-reinstall sentence-transformers huggingface-hub transformers

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings
# Initialize with a simple model like "all-MiniLM-L6-v2" (very popular for sentence embeddings)
emb = HuggingFaceEmbeddings(
    # model_name="sentence-transformers/all-MiniLM-L6-v2"
)

w = emb.embed_query("what is today's date")



/var/folders/hv/m7d19hld69qb83g8n_7x10tc0000gn/T/ipykernel_6517/3572358615.py:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  emb = HuggingFaceEmbeddings(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


# creating a vector database

In [18]:
from langchain.vectorstores import FAISS
#create vector databases
vdb = FAISS.from_documents(documents=df,embedding=emb)

#### for things already in the database : finding similer questiong to the one you asked

In [19]:
#retrieve vecot databases 
pull_vectore =vdb.as_retriever()

In [20]:
questio = pull_vectore.get_relevant_documents("my charger is not working properly")
questio[0]

/var/folders/hv/m7d19hld69qb83g8n_7x10tc0000gn/T/ipykernel_6517/1491808610.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  questio = pull_vectore.get_relevant_documents("my charger is not working properly")


Document(id='71b65672-139b-4614-871d-548a0b4f65ac', metadata={'source': "I'm having an issue with the {product_purchased}. Please assist.\n\nThis one is a little harder to understand but that's just how I am, so I'll leave it at that.\n\nAs the picture above shows I'm using the original charger that came with my {product_purchased}, but it's not charging properly.", 'row': 2665}, page_content="Ticket ID: 2666\nCustomer Name: Lori Smith\nCustomer Email: fischermark@example.org\nCustomer Age: 56\nCustomer Gender: Other\nProduct Purchased: Sony PlayStation\nDate of Purchase: 2021-05-23\nTicket Type: Cancellation request\nTicket Subject: Product compatibility\nTicket Description: I'm having an issue with the {product_purchased}. Please assist.\n\nThis one is a little harder to understand but that's just how I am, so I'll leave it at that.\n\nAs the picture above shows I'm using the original charger that came with my {product_purchased}, but it's not charging properly.\nTicket Status: Pendi

## Asking the AI from the data in the DB

In [21]:
prompt_temp = """
Answer the question based on the context and Question, dont come up with your own answers , 
say i dont know if there is no answer in the here.
context : {context}
QUESTION : {question}
"""
promptt = PromptTemplate(
    template=prompt_temp,input_variables=['context','question']
)

In [22]:
from langchain.chains import RetrievalQA

chain  = RetrievalQA.from_chain_type(llm = llm,
    retriever =  pull_vectore,
    input_key = "query",
    return_source_documents=True,
    chain_type_kwargs = {'prompt' :promptt}
    )

In [23]:
xxx = chain("how munch money did you make last year ")
xxx.get('result')

/var/folders/hv/m7d19hld69qb83g8n_7x10tc0000gn/T/ipykernel_6517/3015519548.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  xxx = chain("how munch money did you make last year ")


"I don't know.  The provided text does not contain information about anyone's income."

In [24]:
xxx = chain("is there someone who complain more than twice , yes or no ")
xxx.get('result')

'No'